<a href="https://colab.research.google.com/github/generalvro/INVEX/blob/main/SE16_PARAMETROS_A_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from datetime import datetime
import pytz
from google.colab import files

# 1. Subir archivo desde el PC
uploaded = files.upload()

# Tomar el nombre del archivo cargado
input_file = list(uploaded.keys())[0]

# 2. Leer archivo Excel
df = pd.read_excel(input_file, dtype=str)

# 3. Normalizar Material (últimos 11 dígitos sin ceros a la izquierda)
def transform_material(s):
    s = '' if s is None else str(s).strip().replace('\xa0', '')
    last11 = s[-11:] if len(s) >= 11 else s
    code = last11.lstrip('0')
    return code if code != '' else '0'

if "Material" in df.columns:
    df["Material"] = df["Material"].apply(transform_material)

# 4. Normalizar cantidades (quitar miles y cambiar coma decimal por punto)
def clean_number(x):
    if pd.isna(x):
        return x
    x = str(x).strip()
    x = x.replace(".", "")     # quita separadores de miles
    x = x.replace(",", ".")    # cambia decimal
    return x

for col in ["Punto pedido", "Stock máximo"]:
    if col in df.columns:
        df[col] = df[col].apply(clean_number)

# 5. Obtener fecha actual en hora de Colombia
tz = pytz.timezone("America/Bogota")
fecha_col = datetime.now(tz).strftime("%d_%m_%Y")

# 6. Exportar con formato SE16_PARAMETROS_DD_MM_YYYY.CSV
output_file = f"SE16_PARAMETROS_{fecha_col}.CSV"
df.to_csv(output_file, index=False, sep=";", encoding="utf-8-sig")

# 7. Descargar el archivo procesado
files.download(output_file)
